In [1]:
#Author: @michaelbrink
#Org: BalloonBox Inc.

In [2]:
from bs4 import BeautifulSoup
import requests
import json
from html.parser import HTMLParser
import urllib.request
import string
import random
import re
import pandas as pd

In [3]:
#Need a JS enabled web service
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import selenium.webdriver.chrome.service as service
from selenium.webdriver.chrome.options import Options
import time

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# Imported the data files

In [5]:
df = pd.read_excel('cleaned_overall_dataset.xlsx',index_col=0)
df.head()

,Rank,Name,Revenue ($M),Revenue % change,Profit ($M),Profit % change,Assets ($M),Market Value ($M),Change in rank (1000),Employees,Change in rank (500),Year,Cat
0,1,Walmart,485873.0,0.8,13643.0,-7.2,198825.0,218619.0,0,2300000,0,2017,No filter
1,2,Berkshire Hathaway,223604.0,6.1,24074.0,0.0,620854.0,411035.0,2,367700,2,2017,No filter
2,3,Apple,215639.0,-7.7,45687.0,-14.4,321686.0,753718.0,0,116000,0,2017,No filter
3,4,Exxon Mobil,205004.0,-16.7,7840.0,-51.5,330314.0,340056.0,-2,72700,-2,2017,No filter
4,5,McKesson,192487.0,6.2,2258.0,53.0,56563.0,31439.0,0,68000,0,2017,No filter


# Visualization

In [ ]:
df.info()

In [ ]:
sns.catplot(x="Market Value ($M)", y="Assets ($M)",kind='swarm', data=df[df["Year"]==2020], aspect=1);

In [ ]:
sns.catplot(x="Revenue ($M)", y="Assets ($M)",kind='swarm', data=df[df["Year"]==2020], aspect=1);

# Modelling prep

In [ ]:
df.head(2)

In [ ]:
df.describe()

In [ ]:
dfPlot = df[(df['Year']== 2019) & (df['Employees'] < 80000) & (df['Profit ($M)'] > -10000)]

In [ ]:
dfPlot = dfPlot.dropna()

In [ ]:
dfPlot.shape

In [ ]:
sns.kdeplot( dfPlot['Profit ($M)'], dfPlot['Employees'],
                 cmap="plasma", shade=True, shade_lowest=False)

In [ ]:
sns.kdeplot( dfPlot['Profit ($M)'], dfPlot['Assets ($M)'],
                 cmap="plasma", shade=True, shade_lowest=False)

In [ ]:
sns.kdeplot( dfPlot['Revenue ($M)'], dfPlot['Market Value ($M)'],
                 cmap="plasma", shade=True, shade_lowest=False)

## Support Vector Machine

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X.describe()

In [ ]:
X = dfPlot.drop(['Rank', 'Name', 'Revenue % change','Profit % change', 'Change in rank (1000)', 'Change in rank (500)', 'Year', 'Assets ($M)'],axis=1)
y = dfPlot['Rank']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(X_train,y_train)

In [ ]:
predictions = lm.predict( X_test)

In [ ]:
plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

In [ ]:
from sklearn import metrics
import numpy as np

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
sns.distplot((y_test-predictions),bins=50);

In [ ]:
coeffecients = pd.DataFrame(lm.coef_,X.columns)
coeffecients.columns = ['Coeffecient']
coeffecients